In [1]:
import os
import shutil
import ants

In [2]:
moving_path = '../data/SigmaAtlas_resized/Sigma_Atlas_mask_resized.nii.gz'
fixed_path = '../data/WHS_SD_rat_atlas_v4_pack/WHS_Atlas_mask_resized.nii.gz'
root_maps = '../data/tissue_maps'

### Register the SIGMA atlas to the WHS atlas

In [3]:
moving_img = ants.image_read(moving_path)
fixed_img = ants.image_read(fixed_path)

In [7]:
out_tx = os.path.join(os.path.dirname(fixed_path), 'tx_')
tx = ants.registration(fixed=fixed_img, moving=moving_img, type_of_transform='Affine', outprefix=out_tx)
warped_moving = ants.apply_transforms(fixed=fixed_img, moving=moving_img, transformlist=tx['fwdtransforms'])
ants.image_write(warped_moving, '../data/SigmaAtlas_resized/sigma_reg.nii.gz')

### Register all the segmentations to WHS space

In [10]:
transform_path = '../data/WHS_SD_rat_atlas_v4_pack/tx_0GenericAffine.mat'
for root, _, files in os.walk(root_maps):
    for name in files:
        print(f'processing file {name}')
        seg_path = os.path.join(root, name)
        out_path = os.path.join(root, 'w2_'+ name)
        moving_seg = ants.image_read(seg_path)
        warped_seg = ants.apply_transforms(fixed=fixed_img, moving=moving_seg, transformlist=transform_path)
        ants.image_write(warped_seg, out_path)

processing file GM-SIGMA-rat3b-2-exVivo.nii.gz
processing file GM-SIGMA-modulated-rat3b-2-exVivo.nii.gz
processing file WM-SIGMA-modulated-rat3b-2-exVivo.nii.gz
processing file WM-SIGMA-rat3b-2-exVivo.nii.gz
processing file GM-SIGMA-Rat3b-1-exVivo.nii.gz
processing file WM-SIGMA-modulated-Rat3b-1-exVivo.nii.gz
processing file WM-SIGMA-Rat3b-1-exVivo.nii.gz
processing file GM-SIGMA-modulated-Rat3b-1-exVivo.nii.gz
processing file GM-SIGMA-modulated-20230224_t2star_02_SNIpost_Exp2.nii.gz
processing file WM-SIGMA-20230224_t2star_02_SNIpost_Exp2.nii.gz
processing file GM-SIGMA-20230224_t2star_02_SNIpost_Exp2.nii.gz
processing file WM-SIGMA-modulated-20230224_t2star_02_SNIpost_Exp2.nii.gz
processing file GM-SIGMA-20230125_t2star_02_SNIpre_Exp2.nii.gz
processing file GM-SIGMA-modulated-20230125_t2star_02_SNIpre_Exp2.nii.gz
processing file WM-SIGMA-20230125_t2star_02_SNIpre_Exp2.nii.gz
processing file WM-SIGMA-modulated-20230125_t2star_02_SNIpre_Exp2.nii.gz


### Sort registered images to different folders

In [11]:
WM_modulated_dir = os.path.join('../data/registered_tissues/WM_modulated')
os.makedirs(WM_modulated_dir, exist_ok = True)

GM_modulated_dir = os.path.join('../data/registered_tissues/GM_modulated')
os.makedirs(GM_modulated_dir, exist_ok = True)

WM_unmodulated_dir = os.path.join('../data/registered_tissues/WM_unmodulated')
os.makedirs(WM_unmodulated_dir, exist_ok = True)

GM_unmodulated_dir = os.path.join('../data/registered_tissues/GM_unmodulated')
os.makedirs(GM_unmodulated_dir, exist_ok = True)


In [12]:
for root, _, files in os.walk(root_maps):
    for name in files:
        if 'w2_WM-SIGMA-modulated' in name:
            shutil.move(os.path.join(root,name), WM_modulated_dir)
        elif 'w2_GM-SIGMA-modulated' in name:
            shutil.move(os.path.join(root,name), GM_modulated_dir)
        elif 'w2_WM-SIGMA' in name:
            shutil.move(os.path.join(root,name), WM_unmodulated_dir)
        elif 'w2_GM-SIGMA' in name:
            shutil.move(os.path.join(root,name), GM_unmodulated_dir)